In [51]:
!pip install numpy
!pip install tensorflow
!pip install spektral
!pip install pandas
!pip install ase
!pip install scipy
!pip install scikit-learn
!pip install matplotlib.pyplot
!pip install pathlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
Looking in indexes: https://pypi.org/simple, 

In [52]:
import numpy as np
import tensorflow as tf
import spektral as sp #pour manip des graphes neuronaux
import pandas as pd
from ase import io
from ase.geometry.analysis import Analysis #va nous permettre de choper angles, liaisons, etc...
from scipy import sparse
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import ECCConv, GlobalSumPool, GraphMasking
from spektral.data import Dataset
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
#connexion au drive où sont mes données
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def get_bonds_length(mol):
  """fonction recupérant la longueur des liaisons entre les atomes => c'est la seule edge feature"""
  bonds=mol.all_bonds
  results=np.zeros((len(bonds[0]),len(bonds[0])))
  for i in range(0,len(bonds[0])):
    for j in range(0,len(bonds[0][i])):
      results[i,j]=mol.get_bond_value(imIdx=0,idxs=[i,j])
  return results.reshape((len(bonds[0]),len(bonds[0]),1))

In [5]:
#creation du dataset
class MyDataset(Dataset):
  def __init__(self):
    super().__init__()
  def read(self):
    graphs=[]
    energies=pd.read_csv('/content/drive/MyDrive/nouveau/energies/train.csv')
    for id in range(1,6771):
      path = '/content/drive/MyDrive/nouveau/atoms/train/id_{}.xyz'.format((id))
      mol=io.read(path)
      analyzed_mol=Analysis(mol)
      a=analyzed_mol.adjacency_matrix #matrice d'ajacence de la molécule au format DOK
      a=a[0].todense() #décompression
      a=a.astype(np.float32)
      na=pd.DataFrame(mol.get_atomic_numbers())
      pos=pd.DataFrame(mol.get_positions())
      x=np.array(pd.concat([na,pos],axis=1)) #matrice des nodes features
      x=x.astype(np.float32)
      e=get_bonds_length(analyzed_mol) #matrice des edge features
      y=energies['energy'][id-1] #label du graph
      graph=sp.data.graph.Graph(x=x,a=a,e=e,y=y)
      graphs.append(graph)
    return graphs

In [6]:
dataset=MyDataset()

In [7]:
dataset

MyDataset(n_graphs=6770)

In [8]:
nf=dataset.n_node_features
ef=dataset.n_edge_features
print(nf,ef)

4 1


In [9]:
#création du modèle
class SomeECC(Model):

  def __init__(self, n_hidden, n_labels):
    super().__init__()
    self.masking=GraphMasking()
    self.conv1=ECCConv(n_hidden,activation='relu')
    self.conv2=ECCConv(n_hidden,activation='relu')
    self.pool=GlobalSumPool()
    #self.dropout=Dropout(0.5)
    self.dense=Dense(n_labels,'linear')

  def call(self, inputs):
    x,a,e=inputs
    x=self.masking(x)
    x=self.conv1([x,a,e])
    x=self.conv2([x,a,e])
    output=self.pool(x)
    output=self.dense(output)
    return output

In [18]:
#init
model=SomeECC(32,dataset.n_labels)
model.compile(optimizer=Adam(learning_rate=1e-3),loss=MeanSquaredError())

In [19]:
#entrainement du ecc
from spektral.data.loaders import BatchLoader

n_epochs=1000
num_graphs_train=int(dataset.n_graphs*0.8)

train_loader=BatchLoader(dataset[:num_graphs_train],batch_size=32,mask=True)
test_dataset=dataset[num_graphs_train:]
test_loader=BatchLoader(test_dataset,batch_size=32,mask=True)

model.fit(train_loader.load(), steps_per_epoch=train_loader.steps_per_epoch, epochs=n_epochs)

Epoch 1/1000


/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


170/170 [==============================] - 3s 11ms/step - loss: 184.7135
Epoch 2/1000
170/170 [==============================] - 2s 11ms/step - loss: 9.5582
Epoch 3/1000
170/170 [==============================] - 2s 12ms/step - loss: 8.4136
Epoch 4/1000
170/170 [==============================] - 2s 11ms/step - loss: 7.8178
Epoch 5/1000
170/170 [==============================] - 2s 12ms/step - loss: 7.3627
Epoch 6/1000
170/170 [==============================] - 2s 12ms/step - loss: 7.3001
Epoch 7/1000
170/170 [==============================] - 2s 12ms/step - loss: 6.8587
Epoch 8/1000
170/170 [==============================] - 2s 12ms/step - loss: 6.8101
Epoch 9/1000
170/170 [==============================] - 2s 12ms/step - loss: 6.6549
Epoch 10/1000
170/170 [==============================] - 2s 12ms/step - loss: 6.4686
Epoch 11/1000
170/170 [==============================] - 2s 12ms/step - loss: 6.3867
Epoch 12/1000
170/170 [==============================] - 2s 13ms/step - loss: 6.7042


In [20]:
print(test_dataset.n_graphs)
loss = model.evaluate(test_loader.load(), steps=test_loader.steps_per_epoch)
print("Done. Test loss: {}".format(loss))

1354


/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


43/43 [==============================] - 0s 7ms/step - loss: 0.0612
Done. Test loss: 0.06115953251719475


In [13]:
#creation du dataset de graph pour le submit
class MySubmitDataset(Dataset):
  def __init__(self):
    super().__init__()
  def read(self):
    graphs=[]
    for id in range(6774,8463): #on prend les indices correspondant à atom_test
      path = '/content/drive/MyDrive/nouveau/atoms/test/id_{}.xyz'.format((id))
      mol=io.read(path)
      analyzed_mol=Analysis(mol)
      a=analyzed_mol.adjacency_matrix #matrice d'ajacence de la molécule au format DOK
      a=a[0].todense() #décompression
      a=a.astype(np.float32)
      na=pd.DataFrame(mol.get_atomic_numbers())
      pos=pd.DataFrame(mol.get_positions())
      x=np.array(pd.concat([na,pos],axis=1))
      x=x.astype(np.float32)
      e=get_bonds_length(analyzed_mol)
      y=0
      graph=sp.data.graph.Graph(x=x,a=a,e=e,y=y)
      graphs.append(graph)
    return graphs

In [35]:
submit_dataset=MySubmitDataset()
print(submit_dataset.n_graphs)
submit_loader=BatchLoader(submit_dataset,batch_size=submit_dataset.n_graphs,mask=True,shuffle=False)

1689


In [58]:
#ptit trick pour compute les valeurs prédites
for batch in submit_loader:
  inputs,_=batch
  pred=model.predict(inputs)
  break

predictions=pd.DataFrame([])
predictions['id']=range(6774,8463)
predictions['predicted']=pred
predictions=predictions.set_index(['id'])
predictions.head()
filepath=Path('/content/drive/MyDrive/predictions.csv')
predictions.to_csv(filepath)

[[-84.72993 ]
 [-84.09983 ]
 [-83.625626]
 ...
 [-67.48208 ]
 [-84.64615 ]
 [-76.48263 ]]
